In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
1,Live Covid-19 News: Political Appointees Meddl...,https://www.nytimes.com/2020/09/12/world/covid...,"Sun, 13 Sep 2020 00:49:00 GMT",2020-09-13 00:49:00+00:00
5,Some college towns grapple with Covid-19 after...,https://www.cnn.com/2020/09/12/health/us-coron...,"Sat, 12 Sep 2020 23:58:00 GMT",2020-09-12 23:58:00+00:00
6,Michigan coronavirus (COVID-19) cases up to 11...,https://www.clickondetroit.com/health/2020/09/...,"Sat, 12 Sep 2020 22:07:00 GMT",2020-09-12 22:07:00+00:00
2,COVID-19 Data Review Panel Examines State's Da...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sat, 12 Sep 2020 21:19:52 GMT",2020-09-12 21:19:52+00:00
4,Governor Cuomo Announces New Record-High Numbe...,https://www.governor.ny.gov/news/governor-cuom...,"Sat, 12 Sep 2020 15:13:21 GMT",2020-09-12 15:13:21+00:00
7,Pfizer and BioNTech announce plan to expand Co...,https://www.statnews.com/2020/09/12/pfizer-and...,"Sat, 12 Sep 2020 15:07:10 GMT",2020-09-12 15:07:10+00:00
0,COVID-19 Daily Update 9-12-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sat, 12 Sep 2020 14:22:35 GMT",2020-09-12 14:22:35+00:00
3,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Fri, 11 Sep 2020 21:30:00 GMT",2020-09-11 21:30:00+00:00
8,COVID-19 Daily Update 9-11-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Fri, 11 Sep 2020 14:10:34 GMT",2020-09-11 14:10:34+00:00
9,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Thu, 10 Sep 2020 21:00:00 GMT",2020-09-10 21:00:00+00:00
